In [6]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [15]:
data = pd.read_csv("./goodreads_data.csv").drop("Unnamed: 0", axis=1)

In [17]:
data.head()

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [19]:
data = data.dropna()
data = data.drop_duplicates()

In [20]:
model = SentenceTransformer('all-MiniLM-L6-v2')

book_description_genres = list()

for i in range(data.shape[0]):
    book_description_genres.append(str(data['Genres'].iloc[i])+str(data['Description'].iloc[i]))

In [23]:
embeddings = model.encode(book_description_genres)

In [25]:
def get_recommendations(book_id, similarity_matrix, k=5):
    similar_books = list(enumerate(similarity_matrix[book_id]))
    similar_books = sorted(similar_books, key=lambda x: x[1], reverse=True)
    similar_books = similar_books[1:k+1]
    recommended_books = [data.iloc[i[0]]["Book"] for i in similar_books]
    return recommended_books

In [26]:
cos_sim = util.cos_sim(embeddings, embeddings)

In [30]:
book_id = 18
recommended_books = get_recommendations(book_id, cos_sim)
print(f"Recommended books for {data.iloc[book_id]['Book']}: {recommended_books}")

Recommended books for The Giving Tree: ['Where the Sidewalk Ends', 'Falling Up', 'A Light in the Attic', 'A Tree Grows in Brooklyn', "A Child's Garden of Verses"]


In [34]:
print("Recommended books for " + str(data.iloc[23]['Book']) + ": " + str(get_recommendations(23, cos_sim)))

Recommended books for Of Mice and Men: ['The Winter of Our Discontent', 'A Day No Pigs Would Die', 'Travels with Charley: In Search of America', 'Cemetery Street', 'Our Mutual Friend']
